In [120]:
import pandas as pd

In [121]:
valeur_ajoutee_gen = pd.read_csv('valeur_ajoutee_gen.csv', low_memory = False).dropna(axis=1, how='all')
valeur_ajoutee_pro = pd.read_csv('valeur_ajoutee_pro.csv', low_memory = False).dropna(axis=1, how='all')

In [122]:
ips_1 = pd.read_csv("ips.csv")
ips_2 = pd.read_csv("fr-en-ips_lycees.csv", delimiter = ';')

In [123]:
ips = pd.concat([ips_1, ips_2], ignore_index=True)
ips = ips.drop(['effectifs_voie_gt', 'effectifs_voie_pro', 'effectifs_ensemble_gt_pro'], axis = 1)
ips = ips.drop_duplicates()
ips

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2022-2023,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,91.5,91.5,NaN,27.2
1,2022-2023,LYON,1.0,AIN,0010006B,LYCEE POLYVALENT SAINT EXUPERY LYCEE DES METIE...,1033.0,VALSERHONE,public,LPO,99.2,84.2,93.6,33.3,24.8
2,2022-2023,LYON,1.0,AIN,0010013J,LYCEE GENERAL LALANDE,1053.0,BOURG EN BRESSE,public,LEGT,123.0,NaN,123.0,34.7,NaN
3,2022-2023,LYON,1.0,AIN,0010016M,LYCEE POLYVALENT JOSEPH MARIE CARRIAT,1053.0,BOURG EN BRESSE,public,LPO,111.7,98.8,109.2,32.9,30.2
4,2022-2023,LYON,1.0,AIN,0010069V,LYCEE GENERAL ET TECHNOLOGIQUE PRIVE ENSEMBLE ...,1034.0,BELLEY,privé sous contrat,LEGT,115.8,NaN,115.8,27.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25372,2018-2019,ROUEN,76.0,SEINE MARITIME,0762953X,LYCEE GENERAL ET TECHNOLOGIQUE DE LA COTE D AL...,76655.0,SAINT VALERY EN CAUX,public,LEGT,108.0,NaN,108.0,NaN,NaN
25373,2018-2019,ROUEN,76.0,SEINE MARITIME,0763112V,LYCEE GENERAL PRIVE LA PROVIDENCE,76259.0,FECAMP,privé sous contrat,LEGT,113.7,NaN,113.7,NaN,NaN
25374,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0770918E,LYCEE POLYVALENT URUGUAY FRANCE LYCEE DES METI...,77014.0,AVON,public,LPO,113.2,87.1,103.7,NaN,NaN
25375,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0770927P,LYCEE GENERAL INTERNATIONAL FRANCOIS 1ER,77186.0,FONTAINEBLEAU,public,LEGT,138.8,NaN,138.8,NaN,NaN


In [124]:
valeur_ajoutee_pro = valeur_ajoutee_pro.drop_duplicates()
valeur_ajoutee_gen = valeur_ajoutee_gen.drop_duplicates()

In [125]:
pourcentage_valeurs_present_gen = (valeur_ajoutee_gen.count()/len(valeur_ajoutee_gen))
inf_seuil_gen = [i for i in range(len(pourcentage_valeurs_present_gen)) if pourcentage_valeurs_present_gen[i] < 0.60]
columns = valeur_ajoutee_gen.columns
print(columns[inf_seuil_gen])

Index(['effectif_presents_serie_stg', 'effectif_presents_serie_sti2d',
       'effectif_presents_serie_std2a', 'effectif_presents_serie_stmg',
       'effectif_presents_serie_sti', 'effectif_presents_serie_stl',
       'effectif_presents_serie_st2s', 'effectif_presents_serie_musiq_danse',
       'effectif_presents_serie_hotellerie', 'taux_brut_de_reussite_serie_stg',
       'taux_brut_de_reussite_serie_sti2d',
       'taux_brut_de_reussite_serie_std2a', 'taux_brut_de_reussite_serie_stmg',
       'taux_brut_de_reussite_serie_sti', 'taux_brut_de_reussite_serie_stl',
       'taux_brut_de_reussite_serie_st2s',
       'taux_brut_de_reussite_serie_musiq_danse',
       'taux_brut_de_reussite_serie_hotellerie',
       'taux_reussite_attendu_acad_serie_l',
       'taux_reussite_attendu_acad_serie_es',
       'taux_reussite_attendu_acad_serie_s',
       'taux_reussite_attendu_acad_serie_stg',
       'taux_reussite_attendu_acad_serie_sti',
       'taux_reussite_attendu_acad_serie_stl',
       'ta

In [126]:
valeur_ajoutee_gen = valeur_ajoutee_gen.drop(columns[inf_seuil_gen], axis = 1)
valeur_ajoutee_gen

,etablissement,annee,ville,code_etablissement,commune,academie,departement,secteur_public_pu_prive_pr,effectif_presents_serie_l,effectif_presents_serie_es,...,libelle_region_2016,code_region_2016,code_departement,libelle_departement,va_reu_total,va_acc_seconde,va_reu_l,va_reu_es,va_reu_s,va_acc_premiere
0,LYCEE ROLLINAT,2012,ARGENTON SUR CREUSE,0360002G,36006,ORLEANS-TOURS,INDRE,public,12.0,20.0,...,CENTRE-VAL DE LOIRE,24,36,Indre,0,-9,1,10,-3,-6
1,LYCEE SAINT GILLES (GENERAL ET TECHNO.),2012,VEIGNE,0371184L,37266,ORLEANS-TOURS,INDRE ET LOIRE,privé sous contrat,NaN,NaN,...,CENTRE-VAL DE LOIRE,24,37,Indre-et-Loire,-8,1,NaN,NaN,NaN,-3
2,LYCEE JEAN MONNET,2012,JOUE LES TOURS,0371417P,37122,ORLEANS-TOURS,INDRE ET LOIRE,public,45.0,139.0,...,CENTRE-VAL DE LOIRE,24,37,Indre-et-Loire,-7,-4,-9,-12,-4,-6
3,LYCEE ISER - BORDIER,2012,GRENOBLE,0380018N,38185,GRENOBLE,ISERE,privé sous contrat,NaN,NaN,...,AUVERGNE-RHONE-ALPES,84,38,Isère,9,-5,NaN,NaN,NaN,-10
4,LYCEE DE LA MATHEYSINE (GENERAL ET TECHNO.),2012,LA MURE D ISERE,0380049X,38269,GRENOBLE,ISERE,public,17.0,17.0,...,AUVERGNE-RHONE-ALPES,84,38,Isère,-1,-1,-3,-6,0,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,LYCEE JEAN MONNET,2015,STRASBOURG,0670078W,67482,STRASBOURG,BAS RHIN,public,33.0,65.0,...,GRAND EST,44,67,Bas-Rhin,-5,5,-1,-3,-8,-1
25496,LYCEE STANISLAS (GENERAL ET TECHNO.),2015,WISSEMBOURG,0670114K,67544,STRASBOURG,BAS RHIN,public,25.0,85.0,...,GRAND EST,44,67,Bas-Rhin,3,10,5,5,2,4
25497,INSTITUTION STE PHILOMENE,2015,HAGUENAU,0671602C,67180,STRASBOURG,BAS RHIN,privé sous contrat,NaN,23.0,...,GRAND EST,44,67,Bas-Rhin,1,-2,NaN,3,1,3
25498,INSTIT.LA DOCTRINE CHRETIENNE,2015,STRASBOURG,0671611M,67482,STRASBOURG,BAS RHIN,privé sous contrat,11.0,20.0,...,GRAND EST,44,67,Bas-Rhin,2,4,2,3,2,5


In [132]:
columnsToDrop = ['effectif_presents_serie_es', 'effectif_presents_serie_s', 'effectif_presents_total_series', 'taux_brut_de_reussite_serie_es',
                    'effectif_de_seconde', 'effectif_de_premiere', 'code_region_2016']
valeur_ajoutee_gen = valeur_ajoutee_gen.drop(columnsToDrop, axis = 1)

In [133]:
valeur_ajoutee_gen

,etablissement,annee,ville,code_etablissement,commune,academie,departement,secteur_public_pu_prive_pr,effectif_presents_serie_l,taux_brut_de_reussite_serie_l,...,taux_acces_attendu_france_premiere_bac,libelle_region_2016,code_departement,libelle_departement,va_reu_total,va_acc_seconde,va_reu_l,va_reu_es,va_reu_s,va_acc_premiere
0,LYCEE ROLLINAT,2012,ARGENTON SUR CREUSE,0360002G,36006,ORLEANS-TOURS,INDRE,public,12.0,83.0,...,89.0,CENTRE-VAL DE LOIRE,36,Indre,0,-9,1,10,-3,-6
1,LYCEE SAINT GILLES (GENERAL ET TECHNO.),2012,VEIGNE,0371184L,37266,ORLEANS-TOURS,INDRE ET LOIRE,privé sous contrat,NaN,NaN,...,88.0,CENTRE-VAL DE LOIRE,37,Indre-et-Loire,-8,1,NaN,NaN,NaN,-3
2,LYCEE JEAN MONNET,2012,JOUE LES TOURS,0371417P,37122,ORLEANS-TOURS,INDRE ET LOIRE,public,45.0,84.0,...,91.0,CENTRE-VAL DE LOIRE,37,Indre-et-Loire,-7,-4,-9,-12,-4,-6
3,LYCEE ISER - BORDIER,2012,GRENOBLE,0380018N,38185,GRENOBLE,ISERE,privé sous contrat,NaN,NaN,...,75.0,AUVERGNE-RHONE-ALPES,38,Isère,9,-5,NaN,NaN,NaN,-10
4,LYCEE DE LA MATHEYSINE (GENERAL ET TECHNO.),2012,LA MURE D ISERE,0380049X,38269,GRENOBLE,ISERE,public,17.0,88.0,...,91.0,AUVERGNE-RHONE-ALPES,38,Isère,-1,-1,-3,-6,0,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,LYCEE JEAN MONNET,2015,STRASBOURG,0670078W,67482,STRASBOURG,BAS RHIN,public,33.0,85.0,...,84.0,GRAND EST,67,Bas-Rhin,-5,5,-1,-3,-8,-1
25496,LYCEE STANISLAS (GENERAL ET TECHNO.),2015,WISSEMBOURG,0670114K,67544,STRASBOURG,BAS RHIN,public,25.0,100.0,...,93.0,GRAND EST,67,Bas-Rhin,3,10,5,5,2,4
25497,INSTITUTION STE PHILOMENE,2015,HAGUENAU,0671602C,67180,STRASBOURG,BAS RHIN,privé sous contrat,NaN,NaN,...,94.0,GRAND EST,67,Bas-Rhin,1,-2,NaN,3,1,3
25498,INSTIT.LA DOCTRINE CHRETIENNE,2015,STRASBOURG,0671611M,67482,STRASBOURG,BAS RHIN,privé sous contrat,11.0,100.0,...,93.0,GRAND EST,67,Bas-Rhin,2,4,2,3,2,5


In [68]:
val_gen['code_etablissement'].value_counts()

0020034B    2
0451442Y    2
0940122X    2
0940118T    2
0932229T    2
           ..
0132828D    1
0133195C    1
0680073K    1
0681761V    1
0671639T    1
Name: code_etablissement, Length: 664, dtype: int64

In [69]:
val_gen[val_gen['code_etablissement'] == "0623981R"]

,etablissement,annee,ville,code_etablissement,commune,academie,departement,secteur_public_pu_prive_pr,taux_brut_de_reussite_serie_s,taux_brut_de_reussite_total_series,...,taux_acces_attendu_acad_premiere_bac,taux_acces_attendu_france_premiere_bac,libelle_region_2016,code_departement,libelle_departement,va_reu_total,va_acc_seconde,va_reu_es,va_reu_s,va_acc_premiere
199,LYCEE VAUBAN (GENERAL ET TECHNO.),2012,AIRE SUR LA LYS,0623981R,62014,LILLE,PAS DE CALAIS,public,89.0,91,...,90.0,91.0,HAUTS-DE-FRANCE,62,Pas-de-Calais,-1,-1,3,-4,-1
24585,LYCEE VAUBAN (GENERAL ET TECHNO.),2015,AIRE SUR LA LYS,0623981R,62014,LILLE,PAS DE CALAIS,public,91.0,96,...,NaN,91.0,HAUTS-DE-FRANCE,62,Pas-de-Calais,2,4,2,-4,0


In [70]:
val_gen

,etablissement,annee,ville,code_etablissement,commune,academie,departement,secteur_public_pu_prive_pr,taux_brut_de_reussite_serie_s,taux_brut_de_reussite_total_series,...,taux_acces_attendu_acad_premiere_bac,taux_acces_attendu_france_premiere_bac,libelle_region_2016,code_departement,libelle_departement,va_reu_total,va_acc_seconde,va_reu_es,va_reu_s,va_acc_premiere
0,LYCEE ROLLINAT,2012,ARGENTON SUR CREUSE,0360002G,36006,ORLEANS-TOURS,INDRE,public,90.0,91,...,86.0,89.0,CENTRE-VAL DE LOIRE,36,Indre,0,-9,10,-3,-6
1,LYCEE SAINT GILLES (GENERAL ET TECHNO.),2012,VEIGNE,0371184L,37266,ORLEANS-TOURS,INDRE ET LOIRE,privé sous contrat,NaN,84,...,78.0,88.0,CENTRE-VAL DE LOIRE,37,Indre-et-Loire,-8,1,NaN,NaN,-3
2,LYCEE JEAN MONNET,2012,JOUE LES TOURS,0371417P,37122,ORLEANS-TOURS,INDRE ET LOIRE,public,91.0,86,...,88.0,91.0,CENTRE-VAL DE LOIRE,37,Indre-et-Loire,-7,-4,-12,-4,-6
3,LYCEE ISER - BORDIER,2012,GRENOBLE,0380018N,38185,GRENOBLE,ISERE,privé sous contrat,NaN,85,...,56.0,75.0,AUVERGNE-RHONE-ALPES,38,Isère,9,-5,NaN,NaN,-10
4,LYCEE DE LA MATHEYSINE (GENERAL ET TECHNO.),2012,LA MURE D ISERE,0380049X,38269,GRENOBLE,ISERE,public,95.0,92,...,92.0,91.0,AUVERGNE-RHONE-ALPES,38,Isère,-1,-1,-6,0,-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25495,LYCEE JEAN MONNET,2015,STRASBOURG,0670078W,67482,STRASBOURG,BAS RHIN,public,79.0,82,...,NaN,84.0,GRAND EST,67,Bas-Rhin,-5,5,-3,-8,-1
25496,LYCEE STANISLAS (GENERAL ET TECHNO.),2015,WISSEMBOURG,0670114K,67544,STRASBOURG,BAS RHIN,public,98.0,99,...,NaN,93.0,GRAND EST,67,Bas-Rhin,3,10,5,2,4
25497,INSTITUTION STE PHILOMENE,2015,HAGUENAU,0671602C,67180,STRASBOURG,BAS RHIN,privé sous contrat,97.0,97,...,NaN,94.0,GRAND EST,67,Bas-Rhin,1,-2,3,1,3
25498,INSTIT.LA DOCTRINE CHRETIENNE,2015,STRASBOURG,0671611M,67482,STRASBOURG,BAS RHIN,privé sous contrat,100.0,100,...,NaN,93.0,GRAND EST,67,Bas-Rhin,2,4,3,2,5


In [81]:
ips['uai'].value_counts()

0010001W    7
0160065J    7
0220023F    7
0212045J    7
0212023K    7
           ..
0851696G    1
0011457D    1
0912433A    1
0301862L    1
0932783V    1
Name: uai, Length: 3698, dtype: int64

In [82]:
ips[ips['uai'] == "0010001W"]

,rentree_scolaire,academie,code_du_departement,departement,uai,nom_de_l_etablissment,code_insee_de_la_commune,nom_de_la_commune,secteur,type_de_lycee,ips_voie_gt,ips_voie_pro,ips_ensemble_gt_pro,ecart_type_de_l_ips_voie_gt,ecart_type_de_l_ips_voie_pro
0,2022-2023,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,91.5,91.5,NaN,27.2
8117,2019-2020,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,90.5,90.5,NaN,29.3
17401,2020-2021,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,88.6,88.6,NaN,29.4
18167,2021-2022,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,91.1,91.1,NaN,30.1
18553,2016-2017,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,89.2,89.2,NaN,NaN
24079,2017-2018,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,89.7,89.7,NaN,NaN
24823,2018-2019,LYON,1.0,AIN,0010001W,LYCEE PROFESSIONNEL ALEXANDRE BERARD LYCEE DES...,1004.0,AMBERIEU EN BUGEY,public,LP,NaN,89.1,89.1,NaN,NaN


In [89]:
val_gen[val_gen["code_etablissement"] == "0160065J"]

,etablissement,annee,ville,code_etablissement,commune,academie,departement,secteur_public_pu_prive_pr,taux_brut_de_reussite_serie_s,taux_brut_de_reussite_total_series,...,taux_acces_attendu_acad_premiere_bac,taux_acces_attendu_france_premiere_bac,libelle_region_2016,code_departement,libelle_departement,va_reu_total,va_acc_seconde,va_reu_es,va_reu_s,va_acc_premiere
4199,LYCEE DE BEAULIEU,2015,COGNAC,0160065J,16102,POITIERS,CHARENTE,privé sous contrat,94.0,95,...,NaN,92.0,NOUVELLE-AQUITAINE,16,Charente,0,-7,2,0,0
